In [34]:
import cv2
import glob
import numpy as np

In [35]:
#display the image
def display_img(img):
    #resize the image
    img = cv2.resize(img,(1200,1000))
    cv2.imshow('img',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
#convert image to grayscales image
def grayscale_cvt(img):
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [36]:
image_paths = glob.glob('../panoramic_images/*.jpg')

In [70]:
#grayscale images
#image_list = [cv2.imread(i,cv2.IMREAD_GRAYSCALE) for i in image_paths ]

#color images
image_list = [cv2.imread(i) for i in image_paths ]

# SIFT features computation


In [71]:
#lets calculate SIFT features for each image and store them in a dict along with image
image_features = {}
sift = cv2.SIFT_create()
for (i,image) in enumerate(image_list):
    keypoints,descriptors = sift.detectAndCompute(image,None)
    image_features['frame'+str(i)] = [image,keypoints,descriptors]
    
    

In [72]:
list(image_features.keys())

['frame0',
 'frame1',
 'frame2',
 'frame3',
 'frame4',
 'frame5',
 'frame6',
 'frame7',
 'frame8',
 'frame9',
 'frame10',
 'frame11']

In [73]:
img1_features = image_features['frame10']
img2_features = image_features['frame4']
#display_img(img1_features[0])

img1,kp1,desc1 = img1_features[0],img1_features[1],img1_features[2]
img2,kp2,desc2 = img2_features[0],img2_features[1],img2_features[2]

In [74]:
#lets visualize keypoints in our image
#display_img(cv2.drawKeypoints(img1,kp1,None))
#display_img(cv2.drawKeypoints(img2,kp2,None))#reference image

# Feature Matching

In [75]:
#create a brute force matcher object
# It will find all of the matching keypoints on two images
bf = cv2.BFMatcher_create(cv2.NORM_L1)

# Find matching points
matches = bf.knnMatch(desc1, desc2,k=2)

In [76]:
#-- Filter matches using the Lowe's ratio test
ratio_thresh = 0.6
good_matches = []
for m,n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)

In [77]:
#-- Draw matches
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
cv2.drawMatches(img1, kp1, img2, kp2, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#-- Show detected matches
display_img(img_matches)

# Homography estimation

In [78]:
good_matches[0].queryIdx
reference_pts = np.array([kp2[match.trainIdx].pt for match in good_matches],dtype = np.float32)
query_pts     = np.array([kp1[match.queryIdx].pt for match in good_matches],dtype = np.float32)

In [83]:
h,status = cv2.findHomography(query_pts,reference_pts,cv2.RANSAC,1.0)

In [84]:
#warp original image into reference plane
display_img(cv2.warpPerspective(img2,h,(img2.shape[0],img2.shape[1])))